In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

# ANSI color codes
RED   = "\033[31m"
BLUE  = "\033[34m"
GREEN = "\033[32m"
YEL   = "\033[33m"
RESET = "\033[0m"

# helper: scientific notation with Unicode superscript exponent
_sup_map = str.maketrans({
    "-": "⁻",
    "0": "⁰",
    "1": "¹",
    "2": "²",
    "3": "³",
    "4": "⁴",
    "5": "⁵",
    "6": "⁶",
    "7": "⁷",
    "8": "⁸",
    "9": "⁹",
})

def sci_sup(x, digits=3):
    """Return string like 3.347×10⁻⁴."""
    if x == 0:
        return f"{0:.{digits}f}×10⁰"
    exp = int(np.floor(np.log10(abs(x))))
    mant = x / 10**exp
    exp_str = str(exp).translate(_sup_map)
    return f"{mant:.{digits}f}×10{exp_str}"

# ODE
def f(x, y):
    return y**2 + 1
